In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
data_dir = '/content/drive/MyDrive/main'
batch_size = 64
img_height = 150
img_width = 150

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    # labels="inferred",
    label_mode="categorical",
    # class_names=None,
    # color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=41,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    # follow_links=False,
)

Found 17676 files belonging to 7 classes.
Using 14141 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    # labels="inferred",
    label_mode="categorical",
    # class_names=None,
    # color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=41,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    # follow_links=False,
)

Found 17676 files belonging to 7 classes.
Using 3535 files for validation.


In [ ]:
from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.009721247 0.8368626


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 7
RN50 = ResNet50(weights= 'imagenet', include_top=False, input_shape= (img_height,img_width,3))
input = RN50.output
input = layers.GlobalAveragePooling2D()(input)
input = layers.Dropout(0.2)(input)
fpred = layers.Dense(num_classes, activation = 'softmax')(input)
model = tf.keras.models.Model(inputs = RN50.input, outputs = fpred)

In [ ]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
!mkdir -p drive/MyDrive/Model/RN50_2/cp

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Model/RN50_2/cp/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history=model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
  callbacks = [model_checkpoint_callback]
)

Epoch 1/100
221/221 [==============================] - 13268s 60s/step - loss: 1.0909 - accuracy: 0.6133 - val_loss: 2.0275 - val_accuracy: 0.6181
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Model/RN50_2/cp/assets
Epoch 2/100
221/221 [==============================] - 41s 188ms/step - loss: 0.2751 - accuracy: 0.9033 - val_loss: 0.8470 - val_accuracy: 0.7330
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Model/RN50_2/cp/assets
Epoch 3/100
221/221 [==============================] - 41s 188ms/step - loss: 0.1740 - accuracy: 0.9417 - val_loss: 0.4490 - val_accuracy: 0.8617
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Model/RN50_2/cp/assets
Epoch 4/100
221/221 [==============================] - 41s 188ms/step - loss: 0.1195 - accuracy: 0.9577 - val_loss: 1.1540 - val_accuracy: 0.7081
Epoch 5/100
221/221 [==============================] - 41s 188ms/step - loss: 0.0933 - accuracy: 0.9686 - val_loss: 0.4271 - val_accuracy: 0.8464
Epoch 6/100
221/221 [=====

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()